In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Flatten, Dense
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import os
from glob import glob
from PIL import Image
from keras.applications import vgg16
from sklearn.model_selection import train_test_split
import zipfile

In [ ]:
base_dir = "/content/sample_data/"
zip_dir = base_dir + "zipImg"
zip_path = base_dir + "zipImg/images.zip"
image_dir = base_dir + "images"
dataset_dir = base_dir + "dataset"
dataset_path = base_dir + 'dataset/dataset.xlsx'

img_df = pd.read_excel(dataset_path)

In [ ]:
os.chdir(base_dir)

if not os.path.exists(zip_dir):
  os.mkdir("zipImg")

if not os.path.exists(image_dir):
  os.mkdir("images")

if not os.path.exists(dataset_dir):
  os.mkdir("dataset")

**Разархивируем архив**

In [ ]:
def zip():
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(image_dir)

**Загружаем изображения**

In [ ]:
def loadPathsImages():
  names = img_df['Путь']
  paths = []

  for name in names:
    full_path = image_dir + "/" + str(name)
    paths.append(full_path)

  paths = np.asarray(paths)
  return paths

In [ ]:
def loadImages(paths):
  images = []

  for path in paths:
    #image = Image.open(path)
    image = cv2.imread(path)

    #print(image.shape)
    #if (len(src.shape) != 3):
      #image = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)


    array = np.array(image)
    images.append(array)

  return images

**Уменьшаем размер до 224x224 и нормализуем**

In [ ]:
def resizeImages(images):
  reImages = []

  for img in images:
    new_img = cv2.resize(img, (224, 224))
    new_img = new_img / 255
    reImages.append(new_img)
  return reImages

**Классы изображений**

In [ ]:
classes = {1:'Спальня', 2:'Частный дом', 3:'Промышленный район', 4:'Кухня', 5:'Гостинная', 6:'Море', 7:'Лес', 8:
 'Дорога', 9:'Здание', 10:'Гора',11:'Равнина', 12:'Улица', 13:'Небоскреб', 14:'Рабочий кабинет', 15:'Магазин'}

**Загружаем метки и кодириуем по OHE**

In [ ]:
def loadLabels():
  df = pd.read_excel(dir)

  labels_df = df['Метка класса']
  labels = []

  for label in labels_df:
    # Метки идут от 1, поэтому - 1
    labels.append(label - 1)

  labels = np.array(labels)
  label_cat = keras.utils.to_categorical(labels)

  return label_cat

**Подготавливаем модель**

In [ ]:
def prepareModel():
  vgg_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

  model = Sequential()
  model.add(vgg_model)
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  # Разблокируем все последние слои для обучения
  vgg_model.trainable = True
  trainable = False
  for layer in vgg_model.layers:
      if layer.name == 'block5_conv1':
          trainable = True
      layer.trainable = trainable

  return model

**Компилируем и обучаем модель**

In [ ]:
def trainModel(model):
  model = prepareModel()

  early_stop = EarlyStopping(monitor='val_acc', min_delta=0.0001,
                           patience=5, verbose=1, mode='auto')

  # filepath = "model.h5"
  # if os.path.exists(filepath):
  #   checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', period=1)
  # else:
  #   checkpoint = ModelCheckpoint(model, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

  # callbacks = [early_stop, checkpoint]

  model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

  # model.fit(X_train, y_train, epochs=5, batch_size=64, callbacks=callbacks)
  model.fit(X_train, y_train, epochs=15, batch_size=512)

  return model

**Оцениваем модель**

In [ ]:
def evaluateModel(model):
  results = model.evaluate(X_test, y_test, batch_size=64)
  print('test loss, test acc:', results)

**Прогноз модели**

In [ ]:
def predict():
  image = Image.open('/content/sample_data/test.jpg')
  image = np.array(image)
  image = cv2.resize(image, (224, 224))
  image = image / 255

  print(X_train[0].shape)
  print(image.shape)
  image = np.expand_dims(image, axis=0)
  pred = model.predict(image)
  print(classes[np.argmax(pred)])

In [ ]:
if len(os.listdir(zip_dir)) == 0:
  zip()

paths = loadPathsImages()
images = loadImages(paths)



X = np.asarray(resizeImages(images))
y = loadLabels()

del paths
del images

print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

del X
del y

In [ ]:
model = trainModel(model)

In [ ]:
evaluateModel(model)